# Laboratorio: Consumiendo un modelo de Machine Learning

**Antes de empezar debemos cerciorarnos que el servidor de la parte anterior esté aún corriendo.**

En este notebook construiremos un cliente muy simple usando la biblioteca `requests` para interactuar con el servidor.

In [2]:
# === 02_client_titanic.ipynb ===
# Cliente para consumir la API del modelo de supervivencia del Titanic

import requests
import json
import pandas as pd

# 1) Configuración básica
BASE_URL = "http://127.0.0.1:8000"   # asegúrate de tener corriendo 01_server_titanic
ENDPOINT  = "/predict"
CONFIDENCE = 0.55                      # opcional: umbral; pon None para usar el del servidor

def build_url(base=BASE_URL, endpoint=ENDPOINT, confidence=CONFIDENCE):
    """Construye la URL con o sin el parámetro confidence."""
    url = base.rstrip("/") + endpoint
    if confidence is not None:
        # requests también permite pasar params=..., pero así ves la URL “final”
        url = f"{url}?confidence={confidence}"
    return url

FULL_URL = build_url()
print("URL:", FULL_URL)

# 2) Función genérica: enviar 1 pasajero y obtener respuesta
def predict_passenger(passenger_dict, url=FULL_URL, verbose=True):
    """
    passenger_dict: dict con las columnas crudas:
      pclass, sex, age, sibsp, parch, fare, embarked
    """
    try:
        resp = requests.post(url, json=passenger_dict, timeout=10)
    except requests.exceptions.RequestException as e:
        raise SystemExit(f"Error de conexión: {e}")

    if verbose:
        print("Status:", resp.status_code)

    if resp.status_code != 200:
        # Deja el detalle del servidor por si hay validaciones que no pasan
        raise ValueError(f"Error {resp.status_code}: {resp.text}")

    data = resp.json()
    # Espera: {"threshold_used":..., "prob_survive":..., "pred_class":..., "pred_label":...}
    if verbose:
        print(f"Umbral usado: {data['threshold_used']}")
        print(f"Prob. sobrevivir: {data['prob_survive']:.3f}")
        print(f"Predicción: {data['pred_class']} → {data['pred_label']}")
    return data

# 3) Ejemplos (ajusta según quieras)
helene = {
    "pclass": 1, "sex": "female", "age": 22, "sibsp": 0, "parch": 1,
    "fare": 80.0, "embarked": "C"
}
giles = {
    "pclass": 2, "sex": "male", "age": 21, "sibsp": 0, "parch": 0,
    "fare": 15.0, "embarked": "S"
}

print("\n--- Helene ---")
pred_h = predict_passenger(helene)

print("\n--- Giles ---")
pred_g = predict_passenger(giles)

# 4) Evaluación por lote (lista de pasajeros → DataFrame con resultados)
def batch_predict(passengers, url=FULL_URL):
    rows = []
    for p in passengers:
        data = predict_passenger(p, url=url, verbose=False)
        rows.append({
            **p,
            "threshold_used": data["threshold_used"],
            "prob_survive":  data["prob_survive"],
            "pred_class":    data["pred_class"],
            "pred_label":    data["pred_label"],
        })
    return pd.DataFrame(rows)

print("\n--- Batch ---")
df_out = batch_predict([helene, giles])
df_out


URL: http://127.0.0.1:8000/predict?confidence=0.55

--- Helene ---
Status: 200
Umbral usado: 0.55
Prob. sobrevivir: 0.960
Predicción: 1 → Sobrevive

--- Giles ---
Status: 200
Umbral usado: 0.55
Prob. sobrevivir: 0.272
Predicción: 0 → No sobrevive

--- Batch ---


,pclass,sex,age,sibsp,parch,fare,embarked,threshold_used,prob_survive,pred_class,pred_label
0,1,female,22,0,1,80.0,C,0.55,0.960318,1,Sobrevive
1,2,male,21,0,0,15.0,S,0.55,0.271564,0,No sobrevive
